# Исследование надежности заемщиков.


**Цель исследования** — проверить четыре гипотезы:
1. Влияет ли семейное положение   на факт погашения кредита в срок.
2. Влияет ли  наличие детей на факт погашения кредита в срок.
3. Влияет ли  наличие образования на факт погашения кредита в срок.
4. Влияет ли  цель получения кредита на факт погашения в срок.


**Ход исследования**

Данные о поведении пользователей находятся в файле `data.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

будут проверены данные на ошибки и оценено их влияние на исследование. Затем, на этапе предобработки будет найдена  возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



# Шаг 1. Обзор данных

In [2]:
import pandas as pd # импорт библиотеки pandas
df = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение в df
df.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов. Тип данных в столбцах — `object`,`int64`, `float64`.

Согласно документации к данным:
* `children` — количество детей;
* `days_employed` — стаж работы;  
* `dob_years` — возраст клиента;
* `education` — образование клиента (категориальная переменная, объектного  типа);
* `education_id` — образование клиента (категориальная переменная, целочисленого типа);
* `family_status` — семейное положение (категориальная переменная, объектного  типа);
* `family_status_id` — семейное положение (категориальная переменная, целочисленного типа);
* `gender` — пол;
* `income_type` — категория занятости;
* `total_income` — ежемесячный доход;
* `purpose ` — цель получения кредита.

В некоторых столбцах видны аномалии, ошибки:
1. В колонке `days_employed` отрицательный стаж работы.
2. В колонке `days_employed` слишком большой положительный стаж у пенсионеров (возможно не дни а часы) 
3. В колонке `total_income` для валюты слишком много незначащих знаков после запятой.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

# Шаг 2.1 Заполнение пропусков

Оценим количество пропусков:

In [4]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски есть только в категории стаж работы и доход. При этом их количество одинаковое.

Оценим общий вес пропусков в данных: 

In [5]:

# отношение всех пропусков в столбце к общему количеству пропусков 
ratio_nan_days_employed = df['days_employed'].isna().sum()/len(df['days_employed']) 
print(f"Пропуски в графе стаж: {ratio_nan_days_employed:.2%}")
ratio_nan_total_income = df['total_income'].isna().sum()/len(df['total_income']) 
print(f"Пропуски в графе доход: {ratio_nan_total_income:.2%}")

Пропуски в графе стаж: 10.10%
Пропуски в графе доход: 10.10%


Десять процентов это весомая доля в выборке, такое количество может существенно повлиять на выводы при анализе.

Проверим связь пропусков с категорией занятости.

Сгруппируем данные дохода по категории занятости:

In [6]:
df.groupby('income_type')['total_income'].count().sort_values(ascending=False) # сортировка по убыванию

income_type
сотрудник          10014
компаньон           4577
пенсионер           3443
госслужащий         1312
безработный            2
студент                1
предприниматель        1
в декрете              1
Name: total_income, dtype: int64

Основу данных составляют 4 типа занятости клиентов - сотрудник, компаньон, пенсионер, госслужащий. Остальные пренебрежимо малы.

Обьеденим категорию предприниматель и компаньон, так это одна и та же категория:

In [7]:
df.loc[df['income_type'] == 'предприниматель','income_type'] = 'компаньон'

Функция для анализа доли пропусков в столбце с выбранной категорией по отношению к общему количеству данных этой категорией: 

In [8]:
# параметрами функцции является столбец - serias, название столбца датафрейма column_str, 
# название категории category в столбце column_str
def analys_ratio_nan(serias, column_str, category):
    ratio_category =  serias.count() / df[column_str].count()
    ratio_nan_category =  serias.isna().sum() / df[column_str].isna().sum() 
    print(f"Доля категории '{category}' составляет: {ratio_category:.2%}")
    print(f'Доля пропусков в  столбце  {column_str} в  категории {category} составляет: {ratio_nan_category:.2%}')

Написание функции для сортировки датафрейма по определенной категории:

In [9]:
# параметрами функции является название столбца и категория в столбце по которой 
# нужно отсортиравать датафрейм
def data_filtred(column_str, category):                           
    try:
        df_category=df[df[column_str] == category]
        return    df_category                         
    except:
        print('Неверный тип данных')

Проверка предположения, что доли пропусков зависят от типа занятости (например не предоставлена подтвержденная информация о стаже и доходе)

In [10]:
# дата фреймы с определенной категорией занятости
workers = data_filtred('income_type','сотрудник')
businessmans = data_filtred('income_type','компаньон')
retirees = data_filtred('income_type','пенсионер')
government_employees = data_filtred('income_type','госслужащий')

# анализ пропусков в каждой категории
analys_ratio_nan(workers['total_income'],'total_income','сотрудник')
analys_ratio_nan(businessmans['total_income'],'total_income','компаньон')
analys_ratio_nan(retirees['total_income'],'total_income','пенсионер')
analys_ratio_nan(government_employees['total_income'],'total_income','госслужащий')

Доля категории 'сотрудник' составляет: 51.75%
Доля пропусков в  столбце  total_income в  категории сотрудник составляет: 50.83%
Доля категории 'компаньон' составляет: 23.66%
Доля пропусков в  столбце  total_income в  категории компаньон составляет: 23.41%
Доля категории 'пенсионер' составляет: 17.79%
Доля пропусков в  столбце  total_income в  категории пенсионер составляет: 19.00%
Доля категории 'госслужащий' составляет: 6.78%
Доля пропусков в  столбце  total_income в  категории госслужащий составляет: 6.76%


Предположение не подтвердилось госслужащие имеют ту же частоту пропусков что и сотрудники.

Для выбора значения которым будем заменять пропуски сравним среднее значение дохода с медианным.

In [11]:
print('Среднее значение в категории доход:', df['total_income'].mean())
print('Медианное значение в категории доход:', df['total_income'].median())

Среднее значение в категории доход: 167422.30220817294
Медианное значение в категории доход: 145017.93753253992


Мы видим разницу в величине дохода. Среднее значение выше медианного за счет влияние клиентов с высокими доходами. По этой причине целесообразней выбрать медианное значение дохода, так как в его расчете нет влияния  отдельных клиентов с высоким доходом. 

Проанализируем медианный доход для каждого типа занятости:

In [12]:
#медианное значение  по типу занятости
workers_median = workers['total_income'].median()
businessmans_median = businessmans['total_income'].median()
retirees_median = retirees['total_income'].median()
government_employees_median = government_employees['total_income'].median()

print('Медианный доход в категории сотрудник:', workers['total_income'].median())
print('Медианный доход в категории компаньон:',businessmans['total_income'].median())
print('Медианный доход в категории пенсионер:',retirees['total_income'].median())
print('Медианный доход в категории госслужащий:',government_employees['total_income'].median())

Медианный доход в категории сотрудник: 142594.39684740017
Медианный доход в категории компаньон: 172396.00084601453
Медианный доход в категории пенсионер: 118514.48641164352
Медианный доход в категории госслужащий: 150447.9352830068


Пенсионеры имеют наименьший доход, а клиенты в категории "компаньон" наоборот обладают 
самыми большими доходами.

Замена пропусков на медианное значение в каждом типе занятости:

In [13]:
# получение столбцов с замененными (на медианное значение) пропусками для основных типов занятости
workers = data_filtred('income_type','сотрудник')['total_income'].fillna(workers_median)
businessmans = data_filtred('income_type','компаньон')['total_income'].fillna(businessmans_median)
retirees = data_filtred('income_type','пенсионер')['total_income'].fillna(retirees_median)
government_employees = data_filtred('income_type','госслужащий')['total_income'].fillna(government_employees_median)

# замена пропусков для каждой категории сотрудников
df['total_income'] = df['total_income'].fillna(workers)
df['total_income'] = df['total_income'].fillna(businessmans)
df['total_income'] = df['total_income'].fillna(retirees)
df['total_income'] = df['total_income'].fillna(government_employees)

Проверим замену в категории `total_income`:

In [14]:
df['total_income'].isna().sum()

0

Замена пропусков в столбце стажа.

Проверка предположения о том, что отрицательные значение стажа работника это стаж до выхода на пенсию.

In [15]:
# подсчет строк с положительным стажем
df[df['days_employed'] > 0].groupby('income_type')['income_type'].count()

income_type
безработный       2
пенсионер      3443
Name: income_type, dtype: int64

Мы видим, что только пенсионеры и безработные имеют положительный стаж, что подтверждает предположение.

Проверим предположение, что у пенсионеров стаж указан в часах

In [16]:
df_positive_days_employed = df[df['days_employed'] > 0] # получение датафрейма с положительным стажем
retirees_old_mean = df_positive_days_employed['dob_years'].mean() # средний возраст таких клиентов
retirees_dob_years_mean = df_positive_days_employed['days_employed'].mean() # средняя величена положительного стажа
print(f'Стаж в годах: {retirees_dob_years_mean/24/365:.1f}')
print(f'Средний возраст клиентов с положительным стажем: {retirees_old_mean:.1f}')

Стаж в годах: 41.7
Средний возраст клиентов с положительным стажем: 59.1


Возраст и стаж соответствует предположению.

Посмотрим на такие же данные, но для отрицательного стажа:

In [17]:
df_negative_days_employed = df[df['days_employed'] < 0] # получение датафрейма с отрицательным стажем
others_old_mean = df_negative_days_employed['dob_years'].mean() # средний возраст таких клиентов
others_dob_years_mean = df_negative_days_employed['days_employed'].mean() # средняя величена отицательного стажа

print(f'Стаж в годах: {others_dob_years_mean/365:.1f}')
print(f'Средний возраст клиентов с отрицательным стажем: {others_old_mean:.1f}')

Стаж в годах: -6.4
Средний возраст клиентов с отрицательным стажем: 39.8


Предположение о том, что отрицательный стаж (в днях) это стаж работающих людей, а положительный (в часах) это стаж пенсионеров, выглядит обоснованно.

Заменим пропуски стажа в категории пенсионеры:

In [18]:
# замена стажа для пенсионеров
retirees_drop_nan = df[df['income_type']=='пенсионер']['days_employed'].fillna(retirees_dob_years_mean)
df['days_employed']=df['days_employed'].fillna(retirees_drop_nan)
# замена стажа для всех остальных категорий
other_drop_nan = df[df['income_type']!='пенсионер']['days_employed'].fillna(others_dob_years_mean)
df['days_employed']=df['days_employed'].fillna(other_drop_nan)

Проверка на количество пропусков:

In [19]:
df['days_employed'].isna().sum()

0

# Шаг 2.2 Проверка данных на аномалии и исправления.

Сгруппируем данные столбца о количестве детей:

In [20]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

У некоторых клиентов количество детей равно 20 при этом в наборе данных отсутствует клиенты со значение от 6 до 19. Это указывает на явную  ошибку в данных.

Замена этого количества детей  на наиболее часто повторяющиеся значение:

In [21]:
children_mode = int(df['children'].mode())
df.loc[df.children == 20,'children'] = children_mode

Проверка замены:

In [22]:
df['children'].value_counts()

 0    14225
 1     4818
 2     2055
 3      330
-1       47
 4       41
 5        9
Name: children, dtype: int64

Проверка предположения  о том, что значение -1 обозначает когда-то наличие детей.

Для этого нужно сравнить средний возраст клиентов у которых в категории `children` стоит -1 со всеми остальными. Так как вероятно те у кого не стало детей будут в среднем старше остальных. 

In [23]:
df[df['children'] == -1]['dob_years'].mean()

42.57446808510638

Средний возраст практически не отличается, возможно, причина в том это дети с некоторыми особенностями, получающие социальную поддержку.

Но вероятней это просто тире.

Замена значений -1 в категории `children` на 1.

In [24]:
df.loc[df.children == -1,'children'] = 1

Анализ всех возрастов клиентов.

In [25]:
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Клиенты с нулевым возрастом это явная аномалия.

Замена клиентов с нулевым возрастом.

Список уникальных категорий занятости для клиентов.

In [26]:
zero_old_unique = df[df['dob_years'] == 0]['income_type'].unique()
zero_old_unique

array(['пенсионер', 'сотрудник', 'компаньон', 'госслужащий'], dtype=object)

Замена нулевого возраста медианным возрастом каждой категории занятости.

In [27]:
for employment in zero_old_unique:
    df.loc[(df['dob_years'] == 0) & (df['income_type'] == employment),'dob_years'] = int(
        data_filtred('income_type', employment)['dob_years'].median()) 

Проверка замены:

In [28]:
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 60, 59, 29, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

Просмотр пола:

In [29]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Две трети заемщиков женщины. Также есть один клиент неопределенного пола. Возможно, что клиент сам себя так идентифицировал, так как при вводе информации  о поле ставят одну букву, а тут не одна буква, поэтому человек заполняющий графу пола не мог ввести три буквы. 

Просмотр этого клиента:

In [30]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Заемщик молод, плюс у него отсутствуют дети. Чаще всего молодёжь  себя идентифицирует полом отличным от мужчины и женщины. Вероятно, клиент сам себя так идентифицировал. Гражданский брак также указывает на это. 

В любом случае один клиент не может повлиять на результаты общих выводов.

# Шаг 2.3. Изменение типов данных.

В категории `total_income` вещественный тип заменим на целочисленный, так как многочисленные знаки после запятой не информативны.

In [31]:
df['total_income'] = df['total_income'].astype('int')

# Шаг 2.4. Удаление дубликатов.

Проверка на наличие неявных дубликатов в столбце `family_status`:

In [32]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Здесь нет неявных дубликатов, но можно привести переменные к одинаковоми стилю

In [33]:
df['family_status'] = df['family_status'].str.lower() # переведем все переменные  в нижний регистр

Проверка категории образования на наличие неявных дубликатов

In [34]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

В переменных  нет выраженного одинаково стиля (присутствуют разные регистры).

Замена на нижний регистр все значения переменных. И проверка на уникальные значения.

In [35]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Проверка на наличние явных дубликатов.

In [36]:
df.duplicated().sum()

71

Удалиние явных дубликатов и проверка.

In [37]:
df = df.drop_duplicates()
df.duplicated().sum()

0

Проверка `purpose` на наличие неявных дубликатов:

In [38]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Среди целей нет абсолютно совпадающих категорий, но некоторые категории смежные.

# Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создание датафрейма словаря для категории `education`:

In [39]:
education_dict = df[['education','education_id']].drop_duplicates().reset_index(drop=True)
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Создание датафрейма словаря для категории `family_status`:

In [40]:
family_dict = df[['family_status','family_status_id']].drop_duplicates().reset_index(drop=True)
family_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Декомпозиция датафрейма путем удаления столбцов образования и семейного статуса

In [41]:
df = df.drop(['family_status','education'],axis=1)

Просмотр датафрейма

In [42]:
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


# Шаг 2.6. Категоризация дохода.

Функция для категоризации дохода

In [43]:
# параметр функции - целочисленный доход income
# возвращаемый параметр - категориальная переменная
def income_group(income):
    try :
        if income <= 30000:
            return 'E'
        if income <= 50000:
            return 'D'
        if income <= 200000:
            return 'C'
        if income <= 1000000:
            return 'B'
        if income >= 1000001:
            return 'A'
    except:
        print('Неверный тип данных')

Создание нового столбца `total_income_category` с категориальной переменной:

In [44]:
df['total_income_category'] = df['total_income'].apply(income_group)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


# Шаг 2.7. Категоризация целей кредита.

Функция, обобщающая цель кредита исходя из большого количества смежных целей:

In [45]:
# параметром функции является строка, функция по вхождению подстроки в строке аргумента возвращает обобщенную категорию
def purpose_operation(purpose):
    if purpose.find('автомобил') >= 0:
        return 'операции с автомобилем'
    if purpose.find('жиль') >= 0  or purpose.find('недвижимост') >= 0:
        return 'операции с недвижимостью'
    if purpose.find('свадьб') >= 0:
        return 'проведение свадьбы'
    if purpose.find('образовани') >= 0:
        return 'получение образования'
    else:
        return 'неустановленая цель'

Добавление нового столбца с обобщенной категорией целей:

In [46]:
df['purpose_category'] = df['purpose'].apply(purpose_operation)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


# Ответы на вопросы.

Сводная таблица зависимости уровня дохода и задолжности:

In [47]:
data_total_income_category = df.pivot_table(index = ['total_income_category'],
                                                   values=['debt','dob_years'],
                                                   aggfunc={'debt':['mean',len],'dob_years':'mean'}
                                                  )
data_total_income_category

debt            dob_years
                           len      mean       mean
total_income_category                              
A                         25.0  0.080000  45.280000
B                       5041.0  0.070621  42.556834
C                      16016.0  0.084915  43.612450
D                        350.0  0.060000  49.537143
E                         22.0  0.090909  55.000000

Крайние категории доходов слишком малочислены, для того чтобы быть опорой для анализа.

На первый взгляд, кажется что лучшими клиентами должны быть клиенты с высокими доходами. Но на самом деле самыми надежными заемщиками (если учитывать только доход) являются клиенты с самым низкими доходами - D (не учитывая категорию E из-за малочисленности).

Возможно это объясняется тем, что категория D более чем в 10 раз меньше B. Также возможно в выборке D больше пенсионеров со стабильным доходом и более низкой долей обязательных трат, например связанных с ипотекой и расходами на детей. 

Сводная таблица зависимости уровня образования и задолжности:

In [51]:
data_education_id.reset_index().replace({'education_id': education_dict.education.to_dict()})

education_id     debt             total_income          
                            len      mean           mean    median
0               высшее   5250.0  0.052952  201475.555238  171485.0
1              среднее  15172.0  0.089902  152742.087068  141695.0
2  неоконченное высшее    744.0  0.091398  178592.478495  155759.0
3            начальное    282.0  0.109929  132581.429078  118941.5
4       ученая степень      6.0  0.000000  174749.833333  157259.5

Самые ненадежные заемщики это заемщики с начальным образованием. Возможно это связано с доходами и той  обязательной долей расходов, которые они вынуждены нести для обеспечения минимальных потребностей.
С уверенностью можно сказать, что надежность заемщиков растет с ростом уровня его образования, исключая людей с ученой степенью, так как их количество слишком мало.
Клиенты с неоконченным высшим образованием по уровню риска практически соответствуют среднему образованию. 

Сводная таблица зависимости количества детей и задолжности.

In [197]:
data_children = df.pivot_table(index=['children'], values=['debt','total_income'],
               aggfunc = {'debt':['mean',len],'total_income':['mean','median','max']})
data_children

debt           total_income                         
              len      mean          max           mean    median
children                                                         
0         14167.0  0.075598    2265604.0  163047.167784  142594.0
1          4855.0  0.091658    2200852.0  169041.779609  145521.0
2          2052.0  0.094542    1103455.0  169747.961014  142594.0
3           330.0  0.081818    1091627.0  179461.130303  145220.0
4            41.0  0.097561     494337.0  166360.707317  150447.0
5             9.0  0.000000     269068.0  167336.888889  168460.0

С ростом количества детей падает надежность заемщиков. Исключением являются клиенты с тремя детьми.

Если посмотреть на среднее значение дохода таких клиентов то это можно объяснить более высокими доходами, но смотря  насколько значительно отличается количество клиентов с двумя и тремя детьми, а также что максимальные их доходы практически не отличаются,при этом отличается среднее от медианного.

 Можем сказать, что это влияние малого размера выборки клиентов с тремя детьми. 

Сводная таблица зависимости семейного положения и задолжности.

In [198]:
display(family_dict)
data_family_status_id = df.pivot_table(index=['family_status_id'], 
                                       values=['debt','total_income','dob_years'],
                           aggfunc={'debt':['mean',len],'total_income':'median','dob_years':'mean'})
data_family_status_id


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


debt            dob_years total_income
                      len      mean       mean       median
family_status_id                                           
0                 12339.0  0.075452  43.715779       142969
1                  4151.0  0.093471  42.272224       142594
2                   959.0  0.065693  56.816475       127310
3                  1195.0  0.071130  45.881172       143450
4                  2810.0  0.097509  38.615658       142594

Самыми ненадежными заемщиками  являются неженатые клиенты. Они же и являются самыми молодыми. Вероятно, потому что при несильно отличающемся (в худшую сторону)  уровне доходов от заемщиков сболее низким долгом, у них присутствуют больше обязательных трат связанных с жильем.

Люди состоящие в гражданском браке имеют меньший долг, чем неженатые заемщики. Но так как так их возраст и доход соответствует женатым. Можем предположить, что причина в личных качествах людей живущих в гражданском браке, но с уверенностью это утверждать нельзя.

Вдовцы/вводы являются самыми надежными заемщиками. Вероятно причина в том, что они бывшие женатые клиенты, которые также имеют низкую задолжность. При этом судя по возрасту являются пенсионерами со стабильными доходами и своим жильем.

Сводная таблица зависимости целей кридита и задолжности.

In [199]:
data_purpose_category = df.pivot_table(index=['purpose_category'], values=['debt','total_income','dob_years'],
               aggfunc={'debt':['mean',len],'total_income':'median','dob_years':'mean'})
data_purpose_category

debt            dob_years total_income
                              len      mean       mean       median
purpose_category                                                   
операции с автомобилем     4306.0  0.093590  43.689039       142594
операции с недвижимостью  10811.0  0.072334  43.361206       142895
получение образования      4013.0  0.092200  43.609768       142594
проведение свадьбы         2324.0  0.080034  43.372203       142594

Операции с недвижимостью имеют самый низкий уровень долга. Вероятно, потому что жилье является первой необходимостью и люди к этому подходят очень серьезно.

Затем  идут клиенты с целью проведения свадьбы эти траты не такие большие как автомобиль. 

Кредиты на авто и на образование имеют примерно одинаковый уровень задолжности. Они не настолько значимы как недвижимость и не столько малы как свадьба. Поэтому задолжность здесь выше.

**Влияние семейного положение на факт погашения долга:**

In [200]:
worst_family_client = data_family_status_id[('debt', 'mean')].max()
best_family_client = data_family_status_id[('debt', 'mean')].min()
print(f'Холостые заемщики являются имеют самый большой показатель долга: {worst_family_client:.2%} невозврата')
print(f'Овдовевшие клиенты имеют самый низкий показатель долга: {best_family_client:.2%} невозврата')

Холостые заемщики являются имеют самый большой показатель долга: 9.75% невозврата
Овдовевшие клиенты имеют самый низкий показатель долга: 6.57% невозврата


Люди состоящие в браке или когда-либо состоявшие имеют низкую вероятность не погашения долга.

**Влияние наличия детей на факт погашения долга**

In [201]:
worst_children_client = data_children.loc[2][('debt', 'mean')]
best_children_client = data_children.loc[0][('debt', 'mean')]
print('С учетом небольших размеров выборки, во всех категориях.')
print(f"Клиенты с двумя детьми,самые ненадежные заемщики: {worst_children_client:.2%} невозврата.\n")
print(f'Самые надежные заемщики - это заемщики без детей: {best_children_client:.2%} невозврата.')

С учетом небольших размеров выборки, во всех категориях.
Клиенты с двумя детьми,самые ненадежные заемщики: 9.45% невозврата.

Самые надежные заемщики - это заемщики без детей: 7.56% невозврата.


Чем больше детей у заемщика тем ниже кредитное качество заемщика.

**Влияние наличие образования на факт погашения долга:**

In [202]:
worst_education_client = data_education_id.loc[3][('debt', 'mean')]
best_education_client = data_education_id.loc[0][('debt', 'mean')]
print(f"Клиенты с начальным образованием самые ненадежные заемщики: {worst_education_client:.2%} невозврата.\n")
print('С учетом небольших размеров выборки, во всех категориях.')
print(f'Самые надежные заемщики - это клиенты с высшим образованием: {best_education_client:.2%} невозврата.')

Клиенты с начальным образованием самые ненадежные заемщики: 10.99% невозврата.

С учетом небольших размеров выборки, во всех категориях.
Самые надежные заемщики - это клиенты с высшим образованием: 5.30% невозврата.


Анализ показал что чем выше уровень образование тем лучше кредитоспособность заемщика.

**Влияние целей получения кредита на факт погашения долга:**

In [203]:
worst_purpose_client = data_purpose_category.loc['операции с автомобилем'][('debt', 'mean')]
best_purpose_client = data_purpose_category.loc['операции с недвижимостью'][('debt', 'mean')]
print(f"Клиенты с целью кредита - 'операции с автомобилем'самые ненадежные заемщики: {worst_purpose_client:.2%} невозврата.")
print(f"Самые надежные заемщики с целью - 'операции с недвижимостью': {best_purpose_client:.2%} невозврата.")

Клиенты с целью кредита - 'операции с автомобилем'самые ненадежные заемщики: 9.36% невозврата.
Самые надежные заемщики с целью - 'операции с недвижимостью': 7.23% невозврата.


Кредиты связанные с недвижимостью и свадьбой являются самыми надежными, вероятно это связано с важностью жилья и небольшой суммой кредита для свадебных расходов. Остальные цели имеют практически одинаковый риск невозврата. 

# Общий вывод:

Все перечисленные факторы влияют на кредитоспособность, при этом в разной степени. Наличие вышего образвания существенно повышает качество заемщика, в отличии от уровня дохода (без учета размера долга). Тем не менее сводные таблицы показывают, что по одному какому-то критерию нельзя с уверенностью делать вывод о кредитоспособности заемщика.